# Task 1: Human recognition using ImageAI

In [88]:
# Area for modules installation


In [73]:
# Importing required modules and setting up environment
import warnings
warnings.filterwarnings('ignore')
from imageai.Detection import ObjectDetection
from IPython.display import Image
import pandas as pd
import os

In [74]:
# Function for counting of number of found people on an image
def human_counter(image_data):
    counter = 0
    for record in image_data:
        if record['name'] == 'person':
            counter+=1
    return counter

In [75]:
# Paths to images
models_path = 'models/'
image_path = 'resources/images/'
image_processed_path = 'resources/images_processed/'

# Getting list of available images
images_available = os.listdir(image_path)
print(images_available)

['3.jpg', '1.jpg', '10.jpg', '7.jpg', '6.jpg', '4.jpg', '5.jpg', '2.jpg', '8.jpg', '9.jpg']


In [76]:
# Setting up RetinaNet detector
detector_RN = ObjectDetection()
detector_RN.setModelTypeAsRetinaNet() 
detector_RN.setModelPath(models_path + 'resnet50_coco_best_v2.0.1.h5') 
detector_RN.loadModel()

# Setting up YOLO detector
detector_YOLO = ObjectDetection()
detector_YOLO.setModelTypeAsYOLOv3() 
detector_YOLO.setModelPath(models_path + 'yolo.h5')  
detector_YOLO.loadModel()

In [77]:
# Array of true number of people on every image
people_true_number = [4, 5, 7, 4, 4, 5, 6, 4, 5, 6]

In [78]:
results_array = []
#results_array.append(['image/model', 'RetinaNet50', 'RetinaNet80', 'YOLO50', 'YOLO80'])

for image_number in range(1,10+1):
    
    # Initializing row array
    temp_array = []

    # Adding image number
    temp_array.append(image_number)
    
    # Adding info about real number of people on the image
    temp_array.append(people_true_number[image_number-1])

    # Using RetinaNet with 50% cutoff
    image_data = detector_RN.detectObjectsFromImage(
    input_image = image_path + str(image_number) + '.jpg',
    output_image_path = image_processed_path + str(image_number) + '_RN_Cutoff_50' + '.jpg',
    minimum_percentage_probability = 50 # Cutoff threshold
    )
    temp_array.append(human_counter(image_data))
    
    # Using RetinaNet with 80% cutoff
    image_data = detector_RN.detectObjectsFromImage(
    input_image = image_path + str(image_number) + '.jpg',
    output_image_path = image_processed_path + str(image_number) + '_RN_Cutoff_80' + '.jpg',
    minimum_percentage_probability = 80 # Cutoff threshold
    )
    temp_array.append(human_counter(image_data))
    
    # Using YOLO with 50% cutoff
    image_data = detector_YOLO.detectObjectsFromImage(
    input_image = image_path + str(image_number) + '.jpg',
    output_image_path = image_processed_path + str(image_number) + '_YOLO_Cutoff_50' + '.jpg',
    minimum_percentage_probability = 50 # Cutoff threshold
    )
    temp_array.append(human_counter(image_data))
    
    # Using YOLO with 80% cutoff
    image_data = detector_YOLO.detectObjectsFromImage(
    input_image = image_path + str(image_number) + '.jpg',
    output_image_path = image_processed_path + str(image_number) + '_YOLO_Cutoff_80' + '.jpg',
    minimum_percentage_probability = 80 # Cutoff threshold
    )
    temp_array.append(human_counter(image_data))
    
    # Adding entire row into resulting array
    results_array.append(temp_array)

2022-05-09 19:03:58.162856: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:502] remapper failed: Invalid argument: Subshape must have computed start >= end since stride is negative, but is 0 and 2 (computed from start 0 and end 9223372036854775807 over shape with rank 2 and stride-1)
2022-05-09 19:03:58.847434: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:502] remapper failed: Invalid argument: Subshape must have computed start >= end since stride is negative, but is 0 and 2 (computed from start 0 and end 9223372036854775807 over shape with rank 2 and stride-1)


In [79]:
# Results visualization
table_columns_names = ['Image number', 'True value', 'RetinaNet50', 'RetinaNet80', 'YOLO50', 'YOLO80']
pd.DataFrame(results_array, columns = table_columns_names)

,Image number,True value,RetinaNet50,RetinaNet80,YOLO50,YOLO80
0,1,4,5,4,4,4
1,2,5,6,1,5,4
2,3,7,5,2,6,5
3,4,4,4,2,4,3
4,5,4,5,4,4,4
5,6,5,5,4,5,5
6,7,6,6,6,6,6
7,8,4,4,4,4,4
8,9,5,5,5,5,5
9,10,6,5,3,4,4


In [80]:
# Creating dictionary for results storage
all_results = {}

for model_type in range(2, 5+1):
    
    # Arranging array for model resutls
    model_results_array = []
    
    # Iterating through rows
    for i in range(0, 9+1):
        
        # Getting current values
        true_value = results_array[i][1]
        predicted_value = results_array[i][model_type]
    
        # Distributing obtained results
        if true_value >= predicted_value:
            TP = predicted_value
            TN = 0
            FP = 0
            FN = true_value - predicted_value
        else:
            TP = true_value
            TN = 0
            FP = predicted_value - true_value
            FN = 0
    
        # Adding metrics to array
        model_results_array.append([TP, TN, FP, FN])
    
    model_results_array.append([sum(row[0] for row in model_results_array),
                                sum(row[1] for row in model_results_array),
                                sum(row[2] for row in model_results_array),
                                sum(row[3] for row in model_results_array)])
    all_results.update({table_columns_names[model_type]: model_results_array})
    
print(all_results)

{'RetinaNet50': [[4, 0, 1, 0], [5, 0, 1, 0], [5, 0, 0, 2], [4, 0, 0, 0], [4, 0, 1, 0], [5, 0, 0, 0], [6, 0, 0, 0], [4, 0, 0, 0], [5, 0, 0, 0], [5, 0, 0, 1], [47, 0, 3, 3]], 'RetinaNet80': [[4, 0, 0, 0], [1, 0, 0, 4], [2, 0, 0, 5], [2, 0, 0, 2], [4, 0, 0, 0], [4, 0, 0, 1], [6, 0, 0, 0], [4, 0, 0, 0], [5, 0, 0, 0], [3, 0, 0, 3], [35, 0, 0, 15]], 'YOLO50': [[4, 0, 0, 0], [5, 0, 0, 0], [6, 0, 0, 1], [4, 0, 0, 0], [4, 0, 0, 0], [5, 0, 0, 0], [6, 0, 0, 0], [4, 0, 0, 0], [5, 0, 0, 0], [4, 0, 0, 2], [47, 0, 0, 3]], 'YOLO80': [[4, 0, 0, 0], [4, 0, 0, 1], [5, 0, 0, 2], [3, 0, 0, 1], [4, 0, 0, 0], [5, 0, 0, 0], [6, 0, 0, 0], [4, 0, 0, 0], [5, 0, 0, 0], [4, 0, 0, 2], [44, 0, 0, 6]]}


In [87]:
# Preparing rows and columns names
result_columns_names = ['TP', 'TN', 'FP', 'FN']
row_indexes = list(range(1, 10+1))
row_indexes.append('sum')

### 1. Displaying results for RetinaNet with threshold of 50%

In [82]:
pd.DataFrame(all_results['RetinaNet50'], columns = result_columns_names, index = row_indexes)

,TP,TN,FP,FN
1,4,0,1,0
2,5,0,1,0
3,5,0,0,2
4,4,0,0,0
5,4,0,1,0
6,5,0,0,0
7,6,0,0,0
8,4,0,0,0
9,5,0,0,0
10,5,0,0,1


### 2. Displaying results for RetinaNet with threshold of 80%

In [83]:
pd.DataFrame(all_results['RetinaNet80'], columns = result_columns_names, index = row_indexes)

,TP,TN,FP,FN
1,4,0,0,0
2,1,0,0,4
3,2,0,0,5
4,2,0,0,2
5,4,0,0,0
6,4,0,0,1
7,6,0,0,0
8,4,0,0,0
9,5,0,0,0
10,3,0,0,3


### 3. Displaying results for YOLO with threshold of 50%

In [84]:
pd.DataFrame(all_results['YOLO50'], columns = result_columns_names, index = row_indexes)

,TP,TN,FP,FN
1,4,0,0,0
2,5,0,0,0
3,6,0,0,1
4,4,0,0,0
5,4,0,0,0
6,5,0,0,0
7,6,0,0,0
8,4,0,0,0
9,5,0,0,0
10,4,0,0,2


### 4. Displaying results for YOLO with threshold of 80%

In [85]:
pd.DataFrame(all_results['YOLO80'], columns = result_columns_names, index = row_indexes)

,TP,TN,FP,FN
1,4,0,0,0
2,4,0,0,1
3,5,0,0,2
4,3,0,0,1
5,4,0,0,0
6,5,0,0,0
7,6,0,0,0
8,4,0,0,0
9,5,0,0,0
10,4,0,0,2


### 5. F-measure calculation for every model

In [86]:
for key, value in all_results.items():
    TP = value[10][0]
    FP = value[10][2]
    FN = value[10][3]
    f_measure = TP / (TP + (FP + FN)/2)
    print('For ' + key + ' f-measure is: ' + str(f_measure))

For RetinaNet50 f-measure is: 0.94
For RetinaNet80 f-measure is: 0.8235294117647058
For YOLO50 f-measure is: 0.9690721649484536
For YOLO80 f-measure is: 0.9361702127659575
